# Top 10k pages data

We're running the top 10k pages from analytics.usa.gov for the last 30 days through the CrUX API to get page-level information.

In [26]:
"""
Loads result to dataframe and analyzes the results
"""

import os
import pandas as pd
from utils.load_results_to_dataframe import load_results_to_dataframe

notebook_dir = os.getcwd()

In [27]:
site_scanning = load_results_to_dataframe(
    file_name="data/site-scanning-weekly-snapshot-20240828.csv",
    notebook_dir=notebook_dir,
)

top_10k = load_results_to_dataframe(
    file_name="data/top-10000-pages-and-screens-30-days-20240828.csv",
    notebook_dir=notebook_dir,
)

top_10k["url"] = top_10k.apply(
    lambda row: "https://" + str(row["domain"]) + str(row["pagePath"]), axis=1
)

top_10k.head()

,page_title,domain,pagePath,pageviews,url
0,National Institute of Standards and Technology...,www.time.gov,/,71938105,https://www.time.gov/
1,National Institute of Standards and Technology...,time.gov,/,66873558,https://time.gov/
2,NWS Radar,radar.weather.gov,/,12205408,https://radar.weather.gov/
3,USPS.com® - USPS Tracking® Results,tools.usps.com,/go/trackconfirmaction,12093137,https://tools.usps.com/go/trackconfirmaction
4,Search Public Sex Offender Registries | Dru Sj...,www.nsopw.gov,/search-public-sex-offender-registries,11036250,https://www.nsopw.gov/search-public-sex-offend...


In [28]:
common_urls = pd.merge(site_scanning, top_10k, left_on="final_url", right_on="url")
common_urls.drop_duplicates(subset="url", keep="first", inplace=True)
common_urls.head()

,target_url,target_url_domain,target_url_top_level_domain,target_url_redirects,final_url,final_url_domain,final_url_top_level_domain,final_url_website,final_url_live,final_url_status_code,...,uswds_string,uswds_string_in_css,uswds_semantic_version,uswds_version,uswds_count,page_title,domain,pagePath,pageviews,url
0,fec.gov,fec.gov,.gov,True,https://www.fec.gov/,fec.gov,gov,www.fec.gov,True,200,...,0,0,NaN,0,40,Home | FEC,www.fec.gov,/,1654,https://www.fec.gov/
1,monahrq.ahrq.gov,ahrq.gov,.gov,True,https://www.ahrq.gov/,ahrq.gov,gov,www.ahrq.gov,True,200,...,0,20,NaN,0,100,Home | Agency for Healthcare Research and Quality,www.ahrq.gov,/,689,https://www.ahrq.gov/
2,www.spoc.spaceforce.mil,spaceforce.mil,.mil,False,https://www.spoc.spaceforce.mil/,spaceforce.mil,mil,www.spoc.spaceforce.mil,True,200,...,0,0,NaN,0,0,Home,www.spoc.spaceforce.mil,/,666,https://www.spoc.spaceforce.mil/
3,www.fema.gov,fema.gov,.gov,False,https://www.fema.gov/,fema.gov,gov,www.fema.gov,True,200,...,39,20,NaN,0,166,Home | FEMA.gov,www.fema.gov,/,26228,https://www.fema.gov/
4,redirector.nlm.nih.gov,nih.gov,.gov,True,https://www.nlm.nih.gov/,nih.gov,gov,www.nlm.nih.gov,True,200,...,11,20,NaN,0,151,National Library of Medicine - National Instit...,www.nlm.nih.gov,/,3054,https://www.nlm.nih.gov/


In [29]:
print(
    f"There are {len(common_urls)} urls from the site scanning "
    f"report that also appear in the top-10000 pages and screens report from analytics.usa.gov."
)

There are 447 urls from the site scanning report that also appear in the top-10000 pages and screens report from analytics.usa.gov.


In [30]:
distinct_domains = top_10k["domain"].unique()
print(
    f"There are {len(distinct_domains)} distinct domains in the top 10k pages report."
)

There are 1253 distinct domains in the top 10k pages report.


In [31]:
crux_sample = load_results_to_dataframe(
    file_name="data/sampled-crux-data-20240828.csv",
    notebook_dir=notebook_dir,
)

crux_sample = crux_sample.dropna(subset=['largest_contentful_paint'])
crux_sample.head(5000)

,page_title,domain,pagePath,pageviews,url,time_to_first_byte,first_contentful_paint,largest_contentful_paint,cumulative_layout_shift,interaction_to_next_paint
0,Diabetes - Search Results - PubMed,pubmed.ncbi.nlm.nih.gov,/,922,https://pubmed.ncbi.nlm.nih.gov/,973.0,1312.0,1397.0,0.05,243.0
1,Live | The White House,www.whitehouse.gov,/live/,17294,https://www.whitehouse.gov/live/,385.0,1073.0,1733.0,0.00,153.0
2,NWS Radar,radar.weather.gov,/station/kfsx/standard,1752,https://radar.weather.gov/station/kfsx/standard,154.0,531.0,1402.0,0.01,112.0
3,Military | USCIS,www.uscis.gov,/military/military,1930,https://www.uscis.gov/military/military,298.0,845.0,1287.0,0.03,131.0
4,Important Visa Information - U.S. Embassy and ...,ng.usembassy.gov,/visas/important-visa-information/,2882,https://ng.usembassy.gov/visas/important-visa-...,1833.0,2601.0,3257.0,0.30,259.0
...,...,...,...,...,...,...,...,...,...,...
1400,NASA Citizen Scientists Spot Object Moving 1 M...,science.nasa.gov,/get-involved/citizen-science/nasa-citizen-sci...,2496,https://science.nasa.gov/get-involved/citizen-...,414.0,1146.0,1129.0,0.00,91.0
1401,"Storm Prediction Center Aug 12, 2024 1630 UTC ...",www.spc.noaa.gov,/products/outlook/day1otlk.html,16285,https://www.spc.noaa.gov/products/outlook/day1...,188.0,548.0,735.0,0.02,313.0
1402,Search Results | InertFinder | Pesticides | US...,ordspub.epa.gov,/ords/pesticides/f,4750,https://ordspub.epa.gov/ords/pesticides/f,2989.0,4811.0,4616.0,0.00,147.0
1403,Southern Region Headquarters,www.weather.gov,/srh/,852,https://www.weather.gov/srh/,815.0,1371.0,1546.0,0.04,402.0


In [32]:
lcp = crux_sample['largest_contentful_paint'] > 2500
cls = crux_sample['cumulative_layout_shift'] > 0.1
inp = crux_sample['interaction_to_next_paint'] > 200

# Filter the dataframe
filtered_df = crux_sample[lcp | cls | inp]
filtered_df.head(5000)

,page_title,domain,pagePath,pageviews,url,time_to_first_byte,first_contentful_paint,largest_contentful_paint,cumulative_layout_shift,interaction_to_next_paint
0,Diabetes - Search Results - PubMed,pubmed.ncbi.nlm.nih.gov,/,922,https://pubmed.ncbi.nlm.nih.gov/,973.0,1312.0,1397.0,0.05,243.0
4,Important Visa Information - U.S. Embassy and ...,ng.usembassy.gov,/visas/important-visa-information/,2882,https://ng.usembassy.gov/visas/important-visa-...,1833.0,2601.0,3257.0,0.30,259.0
5,Log in / NCBI,account.ncbi.nlm.nih.gov,/,24312,https://account.ncbi.nlm.nih.gov/,3613.0,4410.0,4285.0,0.00,404.0
6,Government auctions of seized and surplus prop...,www.usa.gov,/auctions-and-sales,1359,https://www.usa.gov/auctions-and-sales,378.0,686.0,854.0,0.00,246.0
7,"Storm Prediction Center Aug 14, 2024 2000 UTC ...",www.spc.noaa.gov,/products/outlook/day1otlk.html,29356,https://www.spc.noaa.gov/products/outlook/day1...,188.0,548.0,735.0,0.02,313.0
...,...,...,...,...,...,...,...,...,...,...
1397,"Storm Prediction Center Aug 7, 2024 0600 UTC D...",www.spc.noaa.gov,/products/outlook/day1otlk.html,36665,https://www.spc.noaa.gov/products/outlook/day1...,188.0,548.0,735.0,0.02,313.0
1399,No items found - ClinVar - NCBI,www.ncbi.nlm.nih.gov,/clinvar/,34601,https://www.ncbi.nlm.nih.gov/clinvar/,2106.0,3254.0,3602.0,0.19,435.0
1401,"Storm Prediction Center Aug 12, 2024 1630 UTC ...",www.spc.noaa.gov,/products/outlook/day1otlk.html,16285,https://www.spc.noaa.gov/products/outlook/day1...,188.0,548.0,735.0,0.02,313.0
1402,Search Results | InertFinder | Pesticides | US...,ordspub.epa.gov,/ords/pesticides/f,4750,https://ordspub.epa.gov/ords/pesticides/f,2989.0,4811.0,4616.0,0.00,147.0


In [33]:
good_lcp = crux_sample['largest_contentful_paint'] <= 2500
good_cls = crux_sample['cumulative_layout_shift'] <= 0.1
good_inp = crux_sample['interaction_to_next_paint'] <= 200
# Filter the dataframe with the updated conditions
all_good_df = crux_sample[good_lcp & good_cls & good_inp]
all_good_df.head(5000)

,page_title,domain,pagePath,pageviews,url,time_to_first_byte,first_contentful_paint,largest_contentful_paint,cumulative_layout_shift,interaction_to_next_paint
1,Live | The White House,www.whitehouse.gov,/live/,17294,https://www.whitehouse.gov/live/,385.0,1073.0,1733.0,0.00,153.0
2,NWS Radar,radar.weather.gov,/station/kfsx/standard,1752,https://radar.weather.gov/station/kfsx/standard,154.0,531.0,1402.0,0.01,112.0
3,Military | USCIS,www.uscis.gov,/military/military,1930,https://www.uscis.gov/military/military,298.0,845.0,1287.0,0.03,131.0
10,ROME Survey Page | SSA,www.ssa.gov,/myaccount/lp/landing-page-rome.html,18806,https://www.ssa.gov/myaccount/lp/landing-page-...,167.0,708.0,843.0,0.00,70.0
17,"From August 1, 2024, Onward: Entry Requirement...",www.cdc.gov,/importation/dogs/foreign-vaccinated-high-risk...,710,https://www.cdc.gov/importation/dogs/foreign-v...,508.0,1023.0,1024.0,0.00,105.0
...,...,...,...,...,...,...,...,...,...,...
1390,Medical Devices | FDA,www.fda.gov,/medical-devices,8066,https://www.fda.gov/medical-devices,371.0,983.0,1134.0,0.00,113.0
1396,NWS Radar,radar.weather.gov,/station/kmvx/standard,1051,https://radar.weather.gov/station/kmvx/standard,175.0,575.0,1413.0,0.00,142.0
1398,Local Standards: Housing and Utilities | Inter...,www.irs.gov,/businesses/small-businesses-self-employed/loc...,658,https://www.irs.gov/businesses/small-businesse...,341.0,491.0,509.0,0.00,112.0
1400,NASA Citizen Scientists Spot Object Moving 1 M...,science.nasa.gov,/get-involved/citizen-science/nasa-citizen-sci...,2496,https://science.nasa.gov/get-involved/citizen-...,414.0,1146.0,1129.0,0.00,91.0


In [34]:
domain_group = filtered_df.groupby('domain').describe()
domain_group.head(5000)


pageviews                                       \
                             count          mean           std      min   
domain                                                                    
1950census.archives.gov        1.0   6647.000000           NaN   6647.0   
aad.archives.gov              10.0   3453.900000   2340.473525    694.0   
about.usps.com                 5.0  20704.400000  30930.625007    764.0   
account.ncbi.nlm.nih.gov       2.0  13517.500000  15265.728299   2723.0   
ae.usembassy.gov               1.0   2066.000000           NaN   2066.0   
...                            ...           ...           ...      ...   
www.wpc.ncep.noaa.gov          3.0   6365.333333   7743.239524    717.0   
www.wrh.noaa.gov               1.0  35136.000000           NaN  35136.0   
www3.nasa.gov                  2.0   1440.500000     10.606602   1433.0   
yali.state.gov                 2.0   1150.000000    356.381818    898.0   
za.usembassy.gov               2.0   1072.000000    393.151370    794.0   

                                                                \
                               25%      50%       75%      max   
domain                                                           
1950census.archives.gov    6647.00   6647.0   6647.00   6647.0   
aad.archives.gov           2787.25   3278.5   3494.75   9226.0   
about.usps.com             1007.00  12917.0  13954.00  74880.0   
account.ncbi.nlm.nih.gov   8120.25  13517.5  18914.75  24312.0   
ae.usembassy.gov           2066.00   2066.0   2066.00   2066.0   
...                            ...      ...       ...      ...   
www.wpc.ncep.noaa.gov      1952.00   3187.0   9189.50  15192.0   
www.wrh.noaa.gov          35136.00  35136.0  35136.00  35136.0   
www3.nasa.gov              1436.75   1440.5   1444.25   1448.0   
yali.state.gov             1024.00   1150.0   1276.00   1402.0   
za.usembassy.gov            933.00   1072.0   1211.00   1350.0   

                         time_to_first_byte          ...  \
                                      count    mean  ...   
domain                                               ...   
1950census.archives.gov                 1.0   365.0  ...   
aad.archives.gov                       10.0   265.8  ...   
about.usps.com                          5.0   319.2  ...   
account.ncbi.nlm.nih.gov                2.0  2771.5  ...   
ae.usembassy.gov                        1.0  2258.0  ...   
...                                     ...     ...  ...   
www.wpc.ncep.noaa.gov                   3.0   445.0  ...   
www.wrh.noaa.gov                        1.0   679.0  ...   
www3.nasa.gov                           2.0  1842.0  ...   
yali.state.gov                          2.0  3190.5  ...   
za.usembassy.gov                        2.0  1953.5  ...   

                         cumulative_layout_shift        \
                                             75%   max   
domain                                                   
1950census.archives.gov                   0.0100  0.01   
aad.archives.gov                          0.0000  0.00   
about.usps.com                            0.0000  0.00   
account.ncbi.nlm.nih.gov                  0.0000  0.00   
ae.usembassy.gov                          0.9100  0.91   
...                                          ...   ...   
www.wpc.ncep.noaa.gov                     0.2550  0.28   
www.wrh.noaa.gov                          0.0100  0.01   
www3.nasa.gov                             0.0000  0.00   
yali.state.gov                            0.9175  1.11   
za.usembassy.gov                          0.4050  0.45   

                         interaction_to_next_paint                         \
                                             count        mean        std   
domain                                                                      
1950census.archives.gov                        1.0  238.000000        NaN   
aad.archives.gov                              10.0  325.900000   8.569325   
about.usps

In [35]:
all_passing_domain_group = all_good_df.groupby('domain').describe()
all_passing_domain_group.head(5000)

pageviews                                           \
                          count     mean           std      min      25%   
domain                                                                     
2009-2017.state.gov         1.0   1385.0           NaN   1385.0   1385.0   
acis.eoir.justice.gov       2.0  54411.0  71656.786992   3742.0  29076.5   
ais.usvisa-info.com         1.0    829.0           NaN    829.0    829.0   
americorps.gov              1.0  15582.0           NaN  15582.0  15582.0   
aoprals.state.gov           1.0   4954.0           NaN   4954.0   4954.0   
...                         ...      ...           ...      ...      ...   
www.usps.com                1.0   1575.0           NaN   1575.0   1575.0   
www.uspto.gov               1.0   6285.0           NaN   6285.0   6285.0   
www.whitehouse.gov          2.0  30351.0  18465.386484  17294.0  23822.5   
www2.ed.gov                 1.0    803.0           NaN    803.0    803.0   
wwwnc.cdc.gov               2.0    846.0     22.627417    830.0    838.0   

                                                  time_to_first_byte          \
                           50%      75%       max              count    mean   
domain                                                                         
2009-2017.state.gov     1385.0   1385.0    1385.0                1.0   775.0   
acis.eoir.justice.gov  54411.0  79745.5  105080.0                2.0   467.0   
ais.usvisa-info.com      829.0    829.0     829.0                1.0  1801.0   
americorps.gov         15582.0  15582.0   15582.0                1.0   913.0   
aoprals.state.gov       4954.0   4954.0    4954.0                1.0   736.0   
...                        ...      ...       ...                ...     ...   
www.usps.com            1575.0   1575.0    1575.0                1.0   630.0   
www.uspto.gov           6285.0   6285.0    6285.0                1.0   611.0   
www.whitehouse.gov     30351.0  36879.5   43408.0                2.0   341.5   
www2.ed.gov              803.0    803.0     803.0                1.0   663.0   
wwwnc.cdc.gov            846.0    854.0     862.0                2.0   730.0   

                       ... cumulative_layout_shift        \
                       ...                     75%   max   
domain                 ...                                 
2009-2017.state.gov    ...                    0.00  0.00   
acis.eoir.justice.gov  ...                    0.06  0.06   
ais.usvisa-info.com    ...                    0.00  0.00   
americorps.gov         ...                    0.00  0.00   
aoprals.state.gov      ...                    0.00  0.00   
...                    ...                     ...   ...   
www.usps.com           ...                    0.00  0.00   
www.uspto.gov          ...                    0.10  0.10   
www.whitehouse.gov     ...                    0.00  0.00   
www2.ed.gov            ...                    0.00  0.00   
wwwnc.cdc.gov          ...                    0.00  0.00   

                      interaction_to_next_paint                           \
                                          count   mean        std    min   
domain                                                                     
2009-2017.state.gov                         1.0  146.0        NaN  146.0   
acis.eoir.justice.gov                       2.0  141.0   0.000000  141.0   
ais.usvisa-info.com                         1.0   96.0        NaN   96.0   
americorps.gov                              1.0  104.0        NaN  104.0   
aoprals.state.gov                           1.0   71.0        NaN   71.0   
...                                         ...    ...        ...    ...   
www.usps.com                                1.0  179.0        NaN  179.0   
www.uspto.gov                               1.0  113.0        NaN  113.0   
www.whitehouse.gov                          2.0  131.0  31.112698  109.0   
www2.ed.gov                                 1.0   77.0        NaN   77.0   
wwwnc.c